In [1]:
%pdb
%load_ext autotime

Automatic pdb calling has been turned ON


# Preprocessing data 

1. NaN values 
2. Transfrom categorial data 
3. Spliting for test and train 
4. Normalization and stadndartization
5. Feature selection

## Working with NaN 

1. Deleting NaN
2. Imputing Nan

### How to find and delete NaN

In [2]:
import pandas as pd
import numpy as np
from io import StringIO

time: 216 ms


In [3]:
csv_data = '''A,B,C,D
              1.0,2.0,3.0,4.0
              5.0,6.0,,8.0
              10.0,11.0,12.0'''

time: 438 µs


In [8]:
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


time: 7.87 ms


In [9]:
df.isnull() # Return df with True/False varibales describing NaN
df.isnull().sum() # Compute how many NaN in each columns

A    0
B    0
C    1
D    1
dtype: int64

time: 3.27 ms


In [6]:
df.to_numpy() # Convert the df into numpy array (usefull for sklearn, because it works via numpy)

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

time: 2.48 ms


In [7]:
df.dropna() # Drop all rows with NaNs
df.dropna(axis=1) # Drop all columns with NaNs
df.dropna(how='all') # Drop rows if all column has NaN value 
df.dropna(thresh=4) # Drop rows if it has < 4 values 
df.dropna(subset=['C']) # Drop rows if NaN in special columns
df.dropna(subset=['C','D']) # Drop rows if NaN in special columns

,A,B,C,D
0,1.0,2.0,3.0,4.0


time: 16.1 ms


### Imputing NaN variables

In [12]:
from sklearn.impute import SimpleImputer
# docs https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

time: 404 µs


In [11]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean') 
imr = imr.fit(df)
imputed_data = imr.transform(df.to_numpy())
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

time: 6.74 ms


### Some notes about sklearn working principle 
There are 2 basic methods in sklearn classes:

1. Fit 
2. Transform 

**Fit** is a method for calculating or studing parameters from data:
- fit a train sample and calculate weights 
- calculate statistics from data to use it later 
- ...

**Transform** is a method to apply parameters and change our data
- transform data to impute NaN
- ...

![alt](fit_transform_workflow.png)